In [1]:
import numpy as np
import Operator as op
import copy
import MPS

In [2]:
x = MPS.mps.init_rand(d = 2, chi = 5, length = 5)
# :param d: 物理指标维数
# :param chi: 辅助指标截断维数
# :param length: 张量个数

print('MPS中张量形状为')
for n in range(0, x.length):
    print(n, x.tensors[n].shape)
    # tensors是一个list

MPS中张量形状为
0 (2, 5)
1 (5, 2, 5)
2 (5, 2, 5)
3 (5, 2, 5)
4 (5, 2)


In [3]:
tensors = list()
tensors += [np.random.randn(3, 5)]
tensors += [np.random.randn(5, 10, 9)]
tensors += [np.random.randn(9, 2, 7)]
tensors += [np.random.randn(7, 10, 3)]
tensors += [np.random.randn(3, 6)]
y = MPS.mps.init_tensors(tensors)

print('MPS中张量形状为')
for n in range(0, y.length):
    print(n, y.tensors[n].shape)

MPS中张量形状为
0 (3, 5)
1 (5, 10, 9)
2 (9, 2, 7)
3 (7, 10, 3)
4 (3, 6)


In [4]:
for cut_dim in range(5, 1, -1):
    np.random.seed(0)
    x = MPS.mps.init_rand(d = 2, chi = 5, length = 5)
    X = x.mps2tensor()
    X = X/np.linalg.norm(X)
    x.center_orth(center = 2, cut_dim = cut_dim)
    X1 = x.mps2tensor()
    X1 = X1/np.linalg.norm(X1)
    err = np.linalg.norm(X-X1)
    print('裁剪维数 =', cut_dim, '，误差 =', err)

print('\n收缩内向指标后得到单位阵')
I = np.einsum('abc, abd -> cd', x.tensors[1], x.tensors[1])
print(I)

裁剪维数 = 5 ，误差 = 6.175225736672893e-16
裁剪维数 = 4 ，误差 = 6.175225736672893e-16
裁剪维数 = 3 ，误差 = 0.009827143114020739
裁剪维数 = 2 ，误差 = 0.011113769867112433

收缩内向指标后得到单位阵
[[1.00000000e+00 9.35124374e-17]
 [9.35124374e-17 1.00000000e+00]]


In [5]:
print('normalize函数：')
np.random.seed(0)
x = MPS.mps.init_rand(d = 2, chi = 5, length = 5)
print('归一化前 <x|x> =', x.inner(x))
x.normalize()
print('归一化后 <x|x> =', x.inner(x))

print('\ncenter_orth函数：')
np.random.seed(0)
x = MPS.mps.init_rand(d = 2, chi = 5, length = 5)
print('归一化前 <x|x> =', x.inner(x))
x.center_orth(center = 2, normalize = True)
print('归一化后 <x|x> =', x.inner(x))

normalize函数：
归一化前 <x|x> = 24011.680555566854
归一化后 <x|x> = 0.9999999999999983

center_orth函数：
归一化前 <x|x> = 24011.680555566854
归一化后 <x|x> = 1.0


In [6]:
np.random.seed(0)
x = MPS.mps.init_rand(d = 2, chi = 5, length = 5)
y = copy.deepcopy(x)
so = op.spin_operator() # 自旋算符字典
I = np.kron(so['i'], so['i']).reshape(2, 2, 2, 2)   # 单位门
y.evolve_gate(I, 2)

xy = (x.inner(y) ** 2) / (x.inner(x) * y.inner(y))
print('(<x|y>^2)/(<x|x><y|y>) =', xy)

(<x|y>^2)/(<x|x><y|y>) = 0.9999999999999999


In [7]:
x = MPS.mps.init_rand(d = 2, chi = 5, length = 5)
H = op.heisenberg_hamilt([1]*3, [0]*3)
x.TEBD(hamiltonion = H, tau = 0.001, cut_dim = -1, times = 100)
# param hamiltonion: 局域哈密顿量，格点间哈密顿量不同使用list输入，相同直接输入矩阵
# param tau: 模拟时间步长
# param cut_dim: 裁剪维数，-1表示不裁剪
# param times: 模拟步数

In [8]:
H = op.heisenberg_hamilt([1]*3, [0]*3)
gs = MPS.ground_state(length = 5, hamiltonion = H, tau = 0.001, tol = 1e-6, times = 1e6)
# param length: MPS长度（格点数）
# param hamiltonion: 局域哈密顿量
# param tau: 模拟时间步长
# param tol: 误差上限
# param times: 模拟步数上限